In [1]:
import pandas as pd
import numpy as np
import json
from tqdm import tqdm

In [4]:
# List of final 5k products
final_metadata_df = pd.read_csv('../Project_Data/Metadata.csv', index_col=0)
final_asins = list(final_metadata_df.asin)

meta_file = r'E:\ISR Project\meta_Clothing_Shoes_and_Jewelry.json'
file = open(meta_file)
selected_item_lists = []
total_selected_items = 0
for line in tqdm(file):
    item = json.loads(line)
    # If the item in 5k, get all the details of the item
    if item['asin'] in final_asins:
        selected_item_lists.append(item)
        total_selected_items+=1

        if total_selected_items%1000==0:
            print(total_selected_items)
            
data_df = pd.DataFrame(selected_item_lists)
data_df.to_csv('../Project_Data/Metadata_w_urls.csv')

419315it [00:57, 8730.19it/s]

1000


1093940it [02:16, 8787.36it/s]

2000


1674420it [03:24, 8807.96it/s]

...

2103813it [04:14, 8697.48it/s]

4000


2479778it [04:59, 8031.36it/s]

5000


2685059it [05:23, 8289.61it/s]


In [14]:
data_df['imageURLHighRes'].iloc[5:10]

5    [https://images-na.ssl-images-amazon.com/image...
6    [https://images-na.ssl-images-amazon.com/image...
7    [https://images-na.ssl-images-amazon.com/image...
8    [https://images-na.ssl-images-amazon.com/image...
9    [https://images-na.ssl-images-amazon.com/image...
Name: imageURLHighRes, dtype: object

In [2]:
url_list = []
data_df = pd.read_csv('../Project_Data/Metadata_w_urls.csv', index_col=0)

for index, product in tqdm(data_df.iterrows(), total = data_df.shape[0]):
    urls = eval(product['imageURLHighRes'])
    asin = product['asin']

    for ind, url in enumerate(urls):
        extension = url.split('.')[-1]
        file_name = asin+'_'+str(ind)+'.'+extension
        url_list.append((url, file_name))


100%|██████████| 5576/5576 [00:00<00:00, 9829.67it/s]


In [7]:
url_list[:10]

[('https://images-na.ssl-images-amazon.com/images/I/41WNLG9KLhL.jpg',
  'B000072US4_0.jpg'),
 ('https://images-na.ssl-images-amazon.com/images/I/41-05wF3cOL.jpg',
  'B000072US4_1.jpg'),
 ('https://images-na.ssl-images-amazon.com/images/I/31J8LKDaMFL.jpg',
  'B000072US4_2.jpg'),
 ('https://images-na.ssl-images-amazon.com/images/I/41jCPH7z5SL.jpg',
  'B000072US4_3.jpg'),
 ('https://images-na.ssl-images-amazon.com/images/I/41rf15znuxL.jpg',
  'B000072US4_4.jpg'),
 ('https://images-na.ssl-images-amazon.com/images/I/41d9depNfrL.jpg',
  'B000072US4_5.jpg'),
 ('https://images-na.ssl-images-amazon.com/images/I/41KfCQ17c1L.jpg',
  'B000072US4_6.jpg'),
 ('https://images-na.ssl-images-amazon.com/images/I/51BSzCqmqUL.jpg',
  'B000072US4_7.jpg'),
 ('https://images-na.ssl-images-amazon.com/images/I/41s5I7RZKLL.jpg',
  'B000072US4_8.jpg'),
 ('https://images-na.ssl-images-amazon.com/images/I/31wXGYBMt-L.jpg',
  'B000072US4_9.jpg')]

In [4]:
import requests
from multiprocessing.pool import ThreadPool as Pool


import multiprocessing
import os
import requests
import pdb
from multiprocessing  import Pool

url_list_trim = url_list[:20]

IMAGE_FOLDER_HR = '../Project_Data/Images'
if not os.path.exists(IMAGE_FOLDER_HR):
    os.mkdir(IMAGE_FOLDER_HR)

errors = 0

def download(url_params):
    global errors
    url, file_name = url_params
    try:
        r = requests.get(url, allow_redirects=True)
        return (r.content,file_name)
    except:
        errors+=1
        return (-1,-1)


pool = Pool(6)
for content, file_name in tqdm(pool.imap_unordered(download, url_list), total = len(url_list)):
    try:
        open(os.path.join(IMAGE_FOLDER_HR,file_name), 'wb').write(content)
    except:
        continue

100%|██████████| 42108/42108 [1:54:03<00:00,  6.15it/s]  
